In [1]:
import gym
from gym import wrappers
import numpy as np
import io, base64

In [15]:
print(len(gym.envs.registry.all()))

859


# Basics

In [6]:
env = gym.make('MountainCar-v0') #SpaceInvaders-v0, ...
env._max_episode_steps = 200   # set how many steps to terminate the simulation (default 200)
initstate = env.reset()      #returns the init state

# env.render()  #render graphics
env.render() #to applet
screen = env.render(mode='rgb_array')                      #to numpy array (height, width, RGB)
screen = env.render(mode='rgb_array').transpose((2, 0, 1)) #to numpy array (RGB, height, width)
observation, reward, done, info = env.step(1)
env.close()                                                #will close the rendered graphics

In [12]:
print(initstate.size)
print(env.observation_space)
print(env.observation_space.low, env.observation_space.high) # ranges of observation & action
env.observation_space.shape

2
Box(2,)
[-1.2  -0.07] [0.6  0.07]


(2,)

In [10]:
print(env.action_space)
print(env.action_space.n)  #number of possible actions
action = env.action_space.sample()# take a random action

Discrete(3)
3


## [Environment](http://gym.openai.com/envs/)-specific

In [ ]:
# MountainCar-v0
env.goal_position

# Wrapper

In [ ]:
env = wrappers.Monitor(env, "~/gym-results", force=True)  #record video. env has all its usual methods
video = io.open('./gym-results/openaigym.video.%s.video000000.mp4' % env.file_infix, 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''
    <video width="360" height="auto" alt="test" controls><source src="data:video/mp4;base64,{0}" type="video/mp4" /></video>'''
.format(encoded.decode('ascii')))

In [13]:
env = env.unwrapped  # undo the above

True